In [ ]:
#unfinished. dont have enough ram
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sqlalchemy import inspect
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sqlalchemy import create_engine, text, MetaData, Table
import dask.dataframe as dd
from sqlalchemy import inspect
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from joblib import Parallel, delayed




In [2]:
database = '/Users/aryanhazra/Downloads/VSCode Repos/trading_model/src/pre_processing/stock_data/stock_data.db'
engine = create_engine(f'sqlite:///{database}')
inspector = inspect(engine)
tables = inspector.get_table_names()

In [ ]:
#loop over tables and insert them into db
#this is necessary as if I run into ram conflicts, i need to have on db to select randomly from
query = 'SELECT * FROM "{table}"'
for i in tqdm(range(len(tables)), desc="Creating DF"):
    #pd.read_sql(query.format(table=tables[i]), engine).to_sql('stock_data', engine, if_exists='append', index=False)
    df = pd.read_sql(query.format(table=tables[i]), engine)
    # Create 30 new columns for the target (Close price for the next 30 days)
    for i in range(1, 31):
        df[f'Target_{i}'] = df['close'].shift(-i)

    # Get rid of these columns in ticker file, they all end up being null
    df = df.drop(columns=['acos', 'asin'])
    # These columns are too large for this model
    df = df.drop(columns=['cosh', 'exp', 'sinh'])

    # Drop any rows with missing values (i.e., the last 30 rows)
    df = df.dropna()

    df.to_sql('stock_data', engine, if_exists='append', index=False)

In [3]:
# Generate 1,000 unique random row numbers
row_numbers = random.sample(range(11003767), 500000)

# Ensure we do not create empty chunks
num_chunks = min(5, len(row_numbers))  # Use min() to prevent excessive chunking
chunk_size = max(1, len(row_numbers) // num_chunks)  # Avoid division by zero

# Create row chunks
row_chunks = [row_numbers[i * chunk_size:(i + 1) * chunk_size] for i in range(num_chunks)]

# Function to fetch data
def fetch_data(chunk):
    if not chunk:  # Skip empty chunks
        return pd.DataFrame()  
    query = f"SELECT * FROM stock_data WHERE rowid IN ({', '.join(map(str, chunk))})"
    try:
        return pd.read_sql(query, engine, index_col=None)
    except Exception as e:
        print(f"Query failed: {e}")
        return pd.DataFrame()  # Return empty DataFrame if query fails

# Execute queries in parallel with a progress bar
results = []
with ThreadPoolExecutor(max_workers=num_chunks) as executor:
    futures = {executor.submit(fetch_data, chunk): chunk for chunk in row_chunks}
    for future in as_completed(futures):
        try:
            results.append(future.result())
        except Exception as e:
            print(f"Error fetching data: {e}")

# Merge results into a single DataFrame
df = pd.concat(results, ignore_index=True) if results else pd.DataFrame()

# Apply parallel Label Encoding if DataFrame isn't empty
if not df.empty:
    def encode_column(series):
        encoder = LabelEncoder()
        return encoder.fit_transform(series)

    df[['ticker', 'date']] = df[['ticker', 'date']].apply(lambda col: encode_column(col))

In [4]:
# Features: Use technical indicators as features (excluding the 'Close' and 'Target_*' columns)
features = [col for col in df.columns if not col.startswith('Target_')]
X = df[features]

# Target variables: Close prices for the next 30 days
y = df[[f'Target_{i}' for i in range(1, 31)]]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [5]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Predict the target (i.e., the prices for the next 30 days)
y_pred = rf.predict(X_test)

In [ ]:
# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# Store results
comparison_df = pd.DataFrame(y_pred, columns=[f'Predicted_{i}' for i in range(1, 31)])
comparison_df['Actual'] = y_test.values.tolist()


In [ ]:
#drop db after model run
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS stock_data;"))
    conn.commit()  # Required for changes in SQLAlchemy 2.0